In [59]:
# This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [ ]:
import datetime
import pandas as pd
import numpy as np

# Creating the Weekly DataFrame 

In [56]:
def addWeek(df):
    # Creating Last Date Column
    l_date_df = df[['Movie']]
    l_date_df['LastDate'] = pd.to_datetime(df['dateofshow'])
    l_date_df = l_date_df.groupby(
        ['Movie'], as_index=False).aggregate({'LastDate': 'max'})
    df = pd.merge(df, l_date_df, left_on=['Movie'], right_on=[
                  'Movie'], how='left', copy=False)
    df['ReleaseDate'] = pd.to_datetime(df['ReleaseDate'])

    # This is a Monday
    df = df[df['ReleaseDate'] > '2014-03-10'].sort_values(by=['dateofshow'])
    print(max(df['dateofshow']), "   ", min(df['ReleaseDate']))

    int_rep = []
    date = []
    for i in range(1, 1370):
        date.append(datetime.date(2014, 3, 12) +
                    datetime.timedelta(days=i))  # date between
        int_rep.append(i)

    # max(df['dateofshow'])-min(df['ReleaseDate'])  = 1345 Days
    data = {'date': date,
            'int_rep': int_rep
            }
    week_matrix = pd.DataFrame(data)
    week_matrix['week'] = (week_matrix['int_rep']//7)
    week_matrix['date'] = pd.to_datetime(week_matrix['date']).dt.date
    df['dateofshow'] = pd.to_datetime(df['dateofshow'])

    df = df[df['dateofshow'] > '2015-01-5'].sort_values(by=['dateofshow'])
    df['dateofshow'] = pd.to_datetime(df['dateofshow']).dt.date
    df['ReleaseDate'] = pd.to_datetime(df['ReleaseDate']).dt.date
    df['LastDate'] = pd.to_datetime(df['LastDate']).dt.date

    df = pd.merge(df, week_matrix, left_on=[
                  'dateofshow'], right_on=['date'], how='left')
    df = pd.merge(df, week_matrix, left_on=[
                  'ReleaseDate'], right_on=['date'], how='left')
    df = pd.merge(df, week_matrix, left_on=[
                  'LastDate'], right_on=['date'], how='left')

    na_df = df[df.isnull().any(axis=1)]
    # df=df.dropna()

    week_matrix['date'] = pd.to_datetime(week_matrix['date'])
    week_matrix_daywise = week_matrix
    week_matrix = week_matrix.groupby(['week'], as_index=False).aggregate(
        {'date': 'max'})  # commented to get all the dates
    week_matrix['Month'] = week_matrix["date"].dt.month
    # week_matrix.drop(['date'],axis=1,inplace=True)
    # Week _ X same as dateouivalentf show but weak eq

    return df, week_matrix, na_df, week_matrix_daywise

In [57]:
df = pd.read_csv('Daywise_model.csv', index_col=0)
week_df, week_matrix, na_df, week_matrix_daywise = addWeek(df)
week_df['Lifetime'] = week_df['week']-week_df['week_y']
week_df['WeeksSinceRelease'] = week_df['week_x']-week_df['week_y']

week_df = week_df.groupby(['week_x', 'Movie', 'week_y', 'week', 'Lifetime', 'WeeksSinceRelease', 'LastDate', 'ReleaseDate', 'date'], as_index=False).aggregate(
    {'OccAtShow': 'sum', 'OccAtPred': 'sum', 'Capacity': 'sum',
     'Cap_110': 'sum', 'Cap_120': 'sum', 'Cap_131': 'sum', 'Cap_310': 'sum', 'Shows': 'sum', 'BLUSH': 'sum',
     'CARVE': 'sum', 'FRAME': 'sum', 'KITES': 'sum', 'PLUSH': 'sum', 'SPOT': 'sum',
     'STREAK': 'sum', 'WEAVE': 'sum', 'trans_1': 'sum', 'trans_2': 'sum', 'trans_3': 'sum', 'trans_4': 'sum', 'trans_5': 'sum',
     'trans_6': 'sum', 'trans_7': 'sum', 'trans_8': 'sum', 'trans_9': 'sum', 'trans_10': 'sum', 'trans_11': 'sum',
     'trans_12': 'sum', 'dateofshow': 'count'})

week_df = week_df.sort_values(by=['Movie', 'week_x'])

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


2017-11-18     2014-03-14 00:00:00


In [59]:
week_df['OccCumSum'] = week_df.groupby(by=['Movie'], as_index=False)[
    'OccAtShow'].transform(lambda x: x.cumsum()) # Cumulative Sum of Occupancy since the inception of the movie
week_df['CapCumSum'] = week_df.groupby(by=['Movie'], as_index=False)[
    'Capacity'].transform(lambda x: x.cumsum())  # Cumulative Sum of Capacity Allocated to a movie
week_df["OccCumSumPer"] = week_df["OccCumSum"]/week_df["CapCumSum"] # Normalized to percentage
week_df.rename(columns={'week_x': 'WeekOfShow', 'week_y': 'ReleaseWeek',
                        'week': 'LastWeek', 'dateofshow': 'DaysShowedInWeek'}, inplace=True)
mask = week_df.WeeksSinceRelease == 0
week_df.loc[mask, "NewRelease"] = 1 # New Movie Releases
week_df.fillna(0, inplace=True)
new_releases = week_df.groupby(['WeekOfShow'], as_index=False).aggregate({
    'NewRelease': 'sum'})  # Getting Other Movie Releases during the course of the week
week_df = pd.merge(week_df, new_releases, on=['WeekOfShow'], how='left')
week_df.drop(['NewRelease_x'], axis=1, inplace=True)
week_df.rename(columns={'NewRelease_y': 'OtherReleasesInWeek',
                        'OccAtShow': 'OccAtWeek', 'Shows': 'ShowsInWeek'}, inplace=True) # Adding OtherReleasesInWeek as a feature
week_df.sort_values(
    by=['Movie', 'WeekOfShow', 'WeeksSinceRelease'], inplace=True)

week_df['Walkin'] = week_df['OccAtWeek']-week_df['OccAtPred']  # Walkin represent late bookings
week_df['WalkinPer'] = week_df['Walkin']/week_df['Capacity']   # Normalized
week_df['OccPer'] = week_df['OccAtWeek']/week_df['Capacity']   # Normalized
week_df['Target'] = week_df["Lifetime"]-week_df["WeeksSinceRelease"]

# Adding Date to week_df
week_df.drop(['date'], axis=1, inplace=True)
week_df = pd.merge(week_df, week_matrix, left_on=[
                   'WeekOfShow'], right_on=['week'], how='left')

In [ ]:
week_df.to_csv("Week_df.csv")